In [3]:
# importing the necessary packages
import requests
from bs4 import BeautifulSoup
import re
import json
import os

In [17]:
def getReferee(soup1):
    referee = soup1.find_all('div', class_ = "referee")
    for i in referee:
        s = i.get_text()
        match_data['Referee'] = s.strip()

In [18]:
def getStadiumName(soup1):
    stadium =  soup1.find_all('div', class_ = "stadium")
    for i in stadium:
        s = i.get_text()
        match_data['Stadium'] = s.strip()

In [19]:
def getFullTimeScore(soup1):
    match_events = soup1.find_all('div',class_="score fullTime")
    for i in match_events:
        s = i.get_text()
        txt = (s.strip().split('-'))
        match_data['TotalHomeGoals'] = txt[0]
        match_data['TotalAwayGoals'] = txt[1]     

In [20]:
## get the team names which played against each other
def getTeamNames(soup1):
    Name = []
    teamNames = soup1.find_all('span',class_='long')
    for i in teamNames:
        Name.append(i.get_text())
    match_data['HomeTeam'] = Name[0]
    match_data['AwayTeam'] = Name[1]

In [21]:
def getHomeTeamMatchEvents(soup1):
    xyz = (soup1.find_all('div', class_="matchEvents matchEventsContainer"))
    for ptag in xyz:
        for i in ptag.find_all('div', class_="home"):
            for x in i.find_all('div', {'aria-live':'polite'}):
                s = x.text
                txt = (re.sub(r"\s+", "", s)) 
                if re.search("^.*Goal*", txt):
                    match_data['HomeGoalScorer'].append(getGoals(txt))
                elif re.search("^.*Card*", txt):
                    match_data['HomeTeamFouls'].append(getFouls(txt))
                elif re.search("^.*penalty*", txt):
                    match_data["HomeGoalScorer"].append(getPenalty(txt))
                else:
                    print("No match")

In [22]:
def getAwayTeamMatchEvents(soup1):
    xyz = (soup1.find_all('div', class_="matchEvents matchEventsContainer"))
    for ptag in xyz:
        for i in ptag.find_all('div', class_="away"):
            for x in i.find_all('div', {'aria-live':'polite'}):
                s = x.text
                txt = (re.sub(r"\s+", "", s)) 
                if re.search("^.*Goal*", txt):
                    match_data['AwayGoalScorer'].append(getGoals(txt))
                elif re.search("^.*Card*", txt):
                    match_data['AwayTeamFouls'].append(getFouls(txt))
                elif re.search("^.*penalty*", txt):
                    match_data["AwayGoalScorer"].append(getPenalty(txt))
                else:
                    print("No match")

In [23]:
def getGoals(txt):
    goals = {}
    goal = re.findall(r"[^\W\d_]+|\d+",txt)
    goals["Player"] = " ".join(re.findall('[A-Z][^A-Z]*', goal[0]))
    goals["Time"] = goal[1]
    goals["Type"] = "Goal"
    return goals

In [24]:
def getFouls(txt):
    fouls = {}
    foul = re.findall(r"[^\W\d_]+|\d+",txt)
    fouls["Player"] = " ".join(re.findall('[A-Z][^A-Z]*', foul[0]))
    fouls["Time"] = foul[1]
    fouls["Card"] = " ".join(re.findall('[A-Z][^A-Z]*', foul[2]))
    return fouls

In [25]:
def getPenalty(txt):
    Penalties = {}
    Penalty = re.findall(r"[^\W\d_]+|\d+",txt)
    Penalties["Player"] = " ".join(re.findall('[A-Z][^A-Z]*', Penalty[0]))
    Penalties["Time"] = Penalty[1]
    Penalties["Type"] = "Penalty"
    return Penalties

In [26]:
match_data = {
    'HomeGoalScorer':[],
    'AwayGoalScorer':[],
    'HomeTeamFouls':[],
    'AwayTeamFouls':[],
    'HomeTeamAssist':[]
    
}

def getDictionary(match_id):
    url = "https://www.premierleague.com/match/14410"

    r1 = requests.get(url)
    coverpage = r1.content

    soup1 = BeautifulSoup(coverpage, 'html5lib')
    
    getReferee(soup1)
    getStadiumName(soup1)
    getFullTimeScore(soup1)
    getTeamNames(soup1)
    getHomeTeamMatchEvents(soup1)
    getAwayTeamMatchEvents(soup1)
    getHomeAssists(soup1)
    
#     print()
    
#     filename = match_id +'json'
#     with open(os.path.join("/Users/ankitmittal/Desktop/Thesis/Project/data/matchdata/", filename), "w") as outfile:  
#         json.dump(match_data, outfile)
    
    print(match_data)

In [31]:
getDictionary('14410')

{'HomeGoalScorer': [{'Player': 'Héctor Bellerín', 'Time': '8', 'Type': 'Goal'}, {'Player': 'Alexis Sánchez', 'Time': '27', 'Type': 'Goal'}, {'Player': 'Aaron Ramsey', 'Time': '90', 'Type': 'Goal'}, {'Player': 'Héctor Bellerín', 'Time': '8', 'Type': 'Goal'}, {'Player': 'Alexis Sánchez', 'Time': '27', 'Type': 'Goal'}, {'Player': 'Aaron Ramsey', 'Time': '90', 'Type': 'Goal'}], 'AwayGoalScorer': [{'Player': 'Romelu Lukaku', 'Time': '58', 'Type': 'Penalty'}, {'Player': 'Romelu Lukaku', 'Time': '58', 'Type': 'Penalty'}], 'HomeTeamFouls': [{'Player': 'Laurent Koscielny', 'Time': '14', 'Card': 'Red Card'}, {'Player': 'Laurent Koscielny', 'Time': '14', 'Card': 'Red Card'}], 'AwayTeamFouls': [], 'HomeTeamAssist': [{'Time': ['8', '27'], 'Player': 'Danny Welbeck'}, {'Time': ['90', '1'], 'Player': 'MesutÖzil'}], 'Referee': 'Michael Oliver', 'Stadium': 'Emirates Stadium, London', 'TotalHomeGoals': '3', 'TotalAwayGoals': '1', 'HomeTeam': 'Arsenal', 'AwayTeam': 'Everton'}


In [69]:
    url = "https://www.premierleague.com/match/14414"

    r1 = requests.get(url)
    coverpage = r1.content

    soup1 = BeautifulSoup(coverpage, 'html5lib')
    

In [30]:
xyz = (soup1.find_all('div', class_="matchAssistsContainer"))
for ptag in xyz:
       for i in ptag.find_all('div', class_="home"):
            for x in i.find_all('div', class_="event"):
                s = x.text
                txt = (re.sub(r"\s+", "", s)) 
                getAssists(txt)

DannyWelbeck8',27'
MesutÖzil90+1'


In [30]:
def getHomeAssists(soup1):
    xyz = (soup1.find_all('div', class_="matchAssistsContainer"))
    for ptag in xyz:
        for i in ptag.find_all('div', class_="home"):
            for x in i.find_all('div', class_="event"):
                s = x.text
                txt = (re.sub(r"\s+", "", s)) 
                if txt is not None:
                    match_data['HomeTeamAssist'].append(getAssists(txt))
                else:
                    print("No match")

In [29]:
def getAssists(txt):
    assists = {'Time': []}
    assist = re.findall(r"[^\W\d_]+|\d+",txt)
    assists["Player"] = " ".join(re.findall('[A-Z][^A-Z]*', assist[0]))
    for i in range(1,len(assist)):   
        assists['Time'].append(assist[i])
    return assists

In [94]:
# def getGoalKeeper(soup1)
count= 0
x = soup1.find_all('li',{'class':'player','data-position':"0"})
for i in x:
    s = i.text
#     print(s)
    txt = (re.sub(r"\s+", "", s))
    txt1= (re.findall(r"[^\W\d_\-]+|\d+",txt))
#     print((txt1))
#     print(" ".join(re.findall('[A-Z][^A-Z]*', txt1[1])))
    for j in range(1,len(txt1)):
        print(txt1[j])
#     if count == 0:
#         print( txt1 ) 

KasperSchmeichelCGoalkeeper
Ron
RobertZielerGoalkeeper
RyanAllsopGoalkeeper
AaronRamsdaleGoalkeeper
